# Automatic Extraction Method
Here the objective is to automatically extract rules from Instance Space Analysis metadata that can predict if an instance is hard or easy for a given classafier using that instances metafeatures.

## Here are the directories with the datasets we'll be using

In [4]:
# Directories with data for original features, metadata, algorithm bin and beta easy
direc = [#"/home/camila/Documents/Faculdade/Projeto-Mestrado/Dados/Reais/credit-g"
         #,
         "/home/camila/Documents/Faculdade/Projeto-Mestrado/Dados/Reais/compas"
         #,
         #"/home/camila/Documents/Faculdade/Projeto-Mestrado/Dados/Reais/glass",
         #"/home/camila/Documents/Faculdade/Projeto-Mestrado/Dados/Reais/iris",
         #"/home/camila/Documents/Faculdade/Projeto-Mestrado/Dados/Reais/pima_diabetes"
        ]

## Run instance space analysis on train and test datasets

In [7]:
from subprocess import call
import preprocess as pp

# Create train and test datasets (csv files are saved in each directory)
pp.split_sets(direc)

# Run ISA on train and test datasets
for d in direc:
    call(["pyhard", "-F"], cwd = f'{d}/Train')
    call(["pyhard"], cwd = f'{d}/Train')
    call(["pyhard", "-F"], cwd = f'{d}/Test')
    call(["pyhard"], cwd = f'{d}/Test')
    

## Read data and create the sets we'll be using

In [5]:
import preprocess as pp
import os

# Let's process the train and test datasets we'll be using
train_dict = pp.create_sets(direc, "Train")    
test_dict = pp.create_sets(direc, "Test")    

# Here's the format of the dictionary
for key in train_dict:
    print (key)
    for key2 in train_dict[key]:
        print(f"\t {key2}")

original_features
	 compas
meta_features
	 compas
algorithm_bin
	 compas
beta_easy
	 compas


## Let's extract the rule intervals using the train data

In [6]:
import autoextractionmethod as aem

G, B = aem.getIntervals(train_dict)


## Let's see the format of the interval dictionary

In [7]:
# Here's the format of the intervals dictionary
for key in B:
    print (key)
    for key2 in B[key]:
        print(f"\t {key2}")
        for key3 in B[key][key2]:
            print(f"\t \t {key3}")
            for key4 in B[key][key2][key3]:
                print(f"\t \t \t {key4}: {B[key][key2][key3][key4]}")
                

compas
	 bagging
	 	 feature_CL
	 	 	 interval_ind: []
	 	 	 interval_val: []
	 	 feature_CLD
	 	 	 interval_ind: []
	 	 	 interval_val: []
	 	 feature_DCP
	 	 	 interval_ind: [2795, 3693]
	 	 	 interval_val: [0.54, 0.88]
	 	 feature_F1
	 	 	 interval_ind: []
	 	 	 interval_val: []
	 	 feature_Harmfulness
	 	 	 interval_ind: [2713, 3358]
	 	 	 interval_val: [0.0, 0.0]
	 	 feature_N1
	 	 	 interval_ind: [3085, 3693]
	 	 	 interval_val: [1.0, 1.0]
	 	 feature_N2
	 	 	 interval_ind: [2932, 3693]
	 	 	 interval_val: [0.53, 1.0]
	 	 feature_TD_P
	 	 	 interval_ind: []
	 	 	 interval_val: []
	 	 feature_TD_U
	 	 	 interval_ind: []
	 	 	 interval_val: []
	 	 feature_kDN
	 	 	 interval_ind: [2934, 3693]
	 	 	 interval_val: [0.6, 1.0]
	 gradient_boosting
	 	 feature_CL
	 	 	 interval_ind: []
	 	 	 interval_val: []
	 	 feature_CLD
	 	 	 interval_ind: []
	 	 	 interval_val: []
	 	 feature_DCP
	 	 	 interval_ind: [2665, 3693]
	 	 	 interval_val: [0.53, 0.88]
	 	 feature_F1
	 	 	 interval_ind: []
	

## Let's visualize the tables of intervals obtained by the AEM for each dataset

In [7]:
import autoextractionmethod as aem

# Good behavior rules

Good_Rules = {}

for dataset in G:
    Good_Rules[dataset] = aem.simpleRules(G[dataset], 
                                          len(train_dict['original_features'][dataset]))
    
# Bad behavior rules

Bad_Rules = {}

for dataset in B:
    Bad_Rules[dataset] = aem.simpleRules(B[dataset], 
                                          len(train_dict['original_features'][dataset]))

# Display Good and Bad interval tables

for dataset in Good_Rules:
    for algorithm in Good_Rules[dataset]:
        print(f"{dataset} {algorithm}")
        df_good = Good_Rules[dataset][algorithm].sort_values('Support', ascending=False)
        df_bad = Bad_Rules[dataset][algorithm].sort_values('Support', ascending=False)
        
        aem.display_side_by_side(df_good,df_bad, titles=['Good','Bad'])

compas bagging


,Metafeature,Index,Interval,Support
4,Harmfulness,"[1, 2788]","[0.0, 0.0]",0.75
5,N1,"[0, 2373]","[0.0, 0.5]",0.64
2,DCP,"[1, 2264]","[0.12, 0.46]",0.61
9,kDN,"[0, 1570]","[0.0, 0.4]",0.43
0,CL,"[0, 1503]","[0.2, 0.37]",0.41
1,CLD,"[0, 1503]","[0.2, 0.37]",0.41
7,TD_P,"[2320, 3693]","[0.8, 1.0]",0.37
3,F1,"[2461, 3693]","[0.15, 0.38]",0.33
8,TD_U,"[0, 1225]","[0.18, 0.5]",0.33
6,N2,"[730, 1845]","[0.0, 0.0]",0.30


compas gradient_boosting


,Metafeature,Index,Interval,Support
8,TD_U,"[0, 2771]","[0.18, 0.64]",0.75
6,N2,"[0, 2737]","[0.0, 0.5]",0.74
2,DCP,"[0, 2567]","[0.12, 0.53]",0.70
5,N1,"[0, 2538]","[0.0, 0.67]",0.69
9,kDN,"[0, 2376]","[0.0, 0.6]",0.64
3,F1,"[256, 2322]","[0.08, 0.15]",0.56
7,TD_P,"[0, 1822]","[0.6, 0.8]",0.49
4,Harmfulness,"[1478, 2958]","[0.0, 0.0]",0.40
0,CL,"[0, 1268]","[0.2, 0.35]",0.34
1,CLD,"[0, 1268]","[0.2, 0.35]",0.34


compas logistic_regression


...

,Metafeature,Index,Interval,Support
3,F1,"[0, 3428]","[0.0, 0.23]",0.93
6,N2,"[0, 2997]","[0.0, 0.57]",0.81
4,Harmfulness,"[0, 2879]","[0.0, 0.0]",0.78
5,N1,"[0, 2542]","[0.0, 0.67]",0.69
9,kDN,"[0, 2376]","[0.0, 0.6]",0.64
2,DCP,"[0, 2075]","[0.12, 0.46]",0.56
7,TD_P,"[0, 1985]","[0.6, 0.8]",0.54
8,TD_U,"[0, 1896]","[0.18, 0.55]",0.51
0,CL,"[0, 1268]","[0.2, 0.35]",0.34
1,CLD,"[0, 1268]","[0.2, 0.35]",0.34


compas svc_linear


,Metafeature,Index,Interval,Support
5,N1,"[0, 2616]","[0.0, 0.81]",0.71
2,DCP,"[0, 2367]","[0.12, 0.47]",0.64
9,kDN,"[0, 2228]","[0.0, 0.4]",0.60
3,F1,"[256, 2296]","[0.08, 0.15]",0.55
7,TD_P,"[0, 1953]","[0.6, 0.8]",0.53
0,CL,"[0, 1934]","[0.2, 0.42]",0.52
1,CLD,"[0, 1934]","[0.2, 0.42]",0.52
8,TD_U,"[0, 1896]","[0.18, 0.55]",0.51
4,Harmfulness,"[1475, 3072]","[0.0, 0.0]",0.43
6,N2,"[1147, 2737]","[0.0, 0.5]",0.43


compas svc_rbf


,Metafeature,Index,Interval,Support
6,N2,"[0, 2997]","[0.0, 0.57]",0.81
5,N1,"[0, 2575]","[0.0, 0.68]",0.70
2,DCP,"[1, 2489]","[0.12, 0.47]",0.67
3,F1,"[0, 2296]","[0.0, 0.15]",0.62
7,TD_P,"[0, 1819]","[0.6, 0.8]",0.49
0,CL,"[0, 1748]","[0.2, 0.39]",0.47
1,CLD,"[0, 1748]","[0.2, 0.39]",0.47
4,Harmfulness,"[0, 1469]","[0.0, 0.0]",0.40
9,kDN,"[0, 1408]","[0.0, 0.4]",0.38
8,TD_U,"[595, 1896]","[0.41, 0.55]",0.35


## Let's obtain the Positive Rule Disjunction (PRD) and Negative Rule Disjunction (NRD)  

<br>

#### The disjunctive combination of all the simple positive rules or all the simple negative rules is used to obtain a single positive and single negative rule.

In [8]:
import autoextractionmethod as aem

# Get support of PRD and NRD on train data 
aem.RD_support(Good_Rules, Bad_Rules, len(train_dict['original_features'][dataset]))

compas bagging
Good:  1.0 Bad:  0.27 

compas gradient_boosting
Good:  0.8 Bad:  0.32 

compas logistic_regression
Good:  0.99 Bad:  0.27 

compas mlp
Good:  0.83 Bad:  0.27 

compas random_forest
Good:  0.93 Bad:  0.23 

compas svc_linear
Good:  0.83 Bad:  0.27 

compas svc_rbf
Good:  0.81 Bad:  0.37 



## Let's test the test datasets on the rules

In [ ]:
# list of dataset names
dataset_names = list(test_dict['original_features'].keys())

for dataset in dataset_names:
        
        # Let's get the algorithm names for the current dataset
        algorithm_names = list(test_dict['algorithm_bin'][dataset].columns)

        # Let's go through each class (algorithm performance)
        for algorithm in algorithm_names:
            

In [10]:
test_dict['original_features']['compas']

,sex,age,juv_fel_count,juv_misd_count,juv_other_count,priors_count,age_cat_25-45,age_cat_Greaterthan45,age_cat_Lessthan25,race_African-American,race_Caucasian,c_charge_degree_F,c_charge_degree_M
0,0,47,0,0,0,1,0,1,0,0,1,1,0
1,1,40,0,0,0,2,1,0,0,1,0,0,1
2,0,23,0,0,0,0,0,0,1,1,0,1,0
3,1,37,0,0,0,1,1,0,0,0,1,1,0
4,1,29,0,0,0,0,1,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1579,1,35,0,0,0,2,1,0,0,0,1,1,0
1580,1,55,0,0,0,0,0,1,0,0,1,0,1
1581,1,25,0,0,0,0,1,0,0,1,0,1,0
1582,0,27,0,0,0,0,1,0,0,1,0,1,0
